<a href="https://colab.research.google.com/github/LanxyL/Capstone3---Recommendation-System/blob/main/Notebook/Recommendation_System_on_Amazon_home_kitchen_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [3]:
#Read Kitchen&Dining Dataset from last notebook
KitchenDining = pd.read_csv('drive/My Drive/Recommendation_System/KitchenDining80000.csv')

In [4]:
# transform the ratings frame into a ratings matrix
ratings_mtx_df = KitchenDining.pivot_table(values='overall',
                                           index='reviewerID',
                                           columns='asin')
ratings_mtx_df.head(3)

asin,0912696591,B00000JGRP,B00000JGRQ,B00000JGRT,B00002N602,B00004OCIP,B00004OCIQ,B00004OCIU,B00004OCIX,B00004OCIZ,B00004OCJ6,B00004OCJ9,B00004OCJJ,B00004OCJK,B00004OCJN,B00004OCJO,B00004OCJQ,B00004OCJW,B00004OCK0,B00004OCK3,B00004OCKO,B00004OCKR,B00004OCKT,B00004OCL2,B00004OCL8,B00004OCL9,B00004OCLA,B00004OCLK,B00004OCLW,B00004OCM4,B00004OCMB,B00004OCMM,B00004OCMS,B00004OCMX,B00004OCMY,B00004OCNE,B00004OCNJ,B00004OCNK,B00004OCNL,B00004OCNQ,...,B00JW02L9W,B00JWXSPAS,B00JWY1PPE,B00JWYX7YQ,B00JWZCE08,B00JZKZPMO,B00JZL11V2,B00JZXYG9O,B00K11U0Y4,B00K4S1L1K,B00K4V4EWA,B00K6BCV7S,B00K7ILPYU,B00K7P0D9G,B00K8LHA3G,B00KAF5DLQ,B00KAJ3OY0,B00KAK443Y,B00KB7KNRM,B00KBQ1OHQ,B00KBUGQJS,B00KCFDB4U,B00KD83BMS,B00KDV5RZY,B00KFS40N0,B00KI2RQHU,B00KIVD1PC,B00KKYSBUW,B00KM9W2PU,B00KNPVL8W,B00KQCJ0CG,B00KQXDIG4,B00KSRQN0Q,B00KSRSZC0,B00KSRV3UQ,B00KSRXUC0,B00KW1ZDCM,B00KWKQ2MS,B00L8FQZ3M,B00LB18EKK
reviewerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A00473363TJ8YSZ3YAGG9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A00593903MCE5GXTAGGNL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A037357115LCU92V2R0YR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(KitchenDining, test_size=0.2)

In [6]:
print(train.shape)
print(test.shape)
print(KitchenDining.shape)

(65558, 24)
(16390, 24)
(81948, 24)


In [7]:
def evaluate(estimate_f):
  """ RMSE-based predictive performance evaluation with pandas. """
  ids_to_estimate = zip(test.reviewerID, test.asin) 
  estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
  real = test.overall.values
  return np.sqrt(mean_squared_error(real, estimated))

In [8]:
#Predict all reviews as 5 star and see how accurate it would be
def simple_estimate_function(user_id, item_id): return 5

In [9]:
print('RMSE for simple estimate function: %s' % evaluate(simple_estimate_function))

RMSE for simple estimate function: 1.172987612461098


Since we already know that most of the people rate 5 on the review. Let us set this estimation function as a threshold. A better model shall have a much smaller RMSE.

In [10]:
def content_mean(user_id, item_id):
  """ Simple content-filtering based on mean ratings. """
  if user_id in train.reviewerID.to_list():
    user_condition = train.reviewerID == user_id
    return train.loc[user_condition, 'overall'].mean()
  else:
    return 5
print('RMSE for content mean estimation: %s' % evaluate(content_mean))

RMSE for content mean estimation: 1.0235591795439574


In [11]:
def collaborative_mean(user_id, item_id):
  """ Simple collaborative-filtering based on mean ratings. """
  if item_id in train.asin.to_list():
    item_condition = train.asin == item_id
    return train.loc[item_condition, 'overall'].mean()
  else:
    return 5
print('RMSE for collaborative mean estimation: %s' % evaluate(collaborative_mean))

RMSE for collaborative mean estimation: 0.9934427400418299


In [12]:
KitchenDining.head()

,Unnamed: 0,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,salesRank,title,description,related,price,brand,year,month,reviewlength,mainCategory,subCategory1,subCategory2,subCategory3,subCategory4,subCategory5,subCategory6#
0,315,A39RRTO760KGOY,0912696591,"[0, 0]","had no idea about decorating cakes,by this boo...",5.0,follow the instructions,2012-05-27,{'Kitchen & Dining': 9690},Wilton Decorating Cakes Book,This exciting book presents what Wilton has l...,"{'also_bought': ['B003MC22TU', 'B003MC22PY', '...",14.26,Wilton,2012,5,56,Home & Kitchen,Kitchen & Dining,Bakeware,Baking & Cookie Sheets,NaN,NaN,NaN
1,317,AZF13R8HNB59E,0912696591,"[0, 1]","excellent book, shipped on time, packaged nice...",5.0,book,2012-06-12,{'Kitchen & Dining': 9690},Wilton Decorating Cakes Book,This exciting book presents what Wilton has l...,"{'also_bought': ['B003MC22TU', 'B003MC22PY', '...",14.26,Wilton,2012,6,24,Home & Kitchen,Kitchen & Dining,Bakeware,Baking & Cookie Sheets,NaN,NaN,NaN
2,318,A1KCBA3PKLV2IT,0912696591,"[0, 0]",i loved it. don't use it much but when i had i...,5.0,great!,2012-12-26,{'Kitchen & Dining': 9690},Wilton Decorating Cakes Book,This exciting book presents what Wilton has l...,"{'also_bought': ['B003MC22TU', 'B003MC22PY', '...",14.26,Wilton,2012,12,27,Home & Kitchen,Kitchen & Dining,Bakeware,Baking & Cookie Sheets,NaN,NaN,NaN
3,321,A3JM7FVJMPJ1XD,0912696591,"[4, 4]",So I actually bought this thinking it was the ...,3.0,Good for what it's worth,2012-08-16,{'Kitchen & Dining': 9690},Wilton Decorating Cakes Book,This exciting book presents what Wilton has l...,"{'also_bought': ['B003MC22TU', 'B003MC22PY', '...",14.26,Wilton,2012,8,92,Home & Kitchen,Kitchen & Dining,Bakeware,Baking & Cookie Sheets,NaN,NaN,NaN
4,326,A1B9BZ3AJ5V1NJ,0912696591,"[0, 0]",I'm hoping that I can learn how to make gum pa...,5.0,learning,2014-05-21,{'Kitchen & Dining': 9690},Wilton Decorating Cakes Book,This exciting book presents what Wilton has l...,"{'also_bought': ['B003MC22TU', 'B003MC22PY', '...",14.26,Wilton,2014,5,24,Home & Kitchen,Kitchen & Dining,Bakeware,Baking & Cookie Sheets,NaN,NaN,NaN


In [13]:
import json
product_related = []
for i in range(KitchenDining.shape[0]):
  product_related.append([KitchenDining.asin[i],KitchenDining.related[i]])
product_related = pd.DataFrame(product_related)
product_related = product_related.rename(columns={0:'asin', 1:'related'})
product_related.dropna(inplace=True)
product_related.drop_duplicates(inplace=True)
product_related.reset_index()
product_related.head()

,asin,related
0,0912696591,"{'also_bought': ['B003MC22TU', 'B003MC22PY', '..."
41,B00000JGRP,"{'also_viewed': ['B0000645YL', 'B0000645YM', '..."
68,B00000JGRQ,"{'also_bought': ['B00004S9CJ', 'B001D9B2R2', '..."
80,B00000JGRT,"{'also_bought': ['B00004S9D3', '0894803123', '..."
152,B00002N602,"{'also_bought': ['B000ORD8GM', '0965410803', '..."


In [14]:
product_related.shape

(3320, 2)

In [15]:
relation_dict = dict()
for i in product_related.index:
  x=product_related.related[i]
  x = str(x).replace("\'", "\"")
  dictx = json.loads(x)
  relation_dict[product_related.asin[i]] = dictx


In [16]:
product_related = pd.DataFrame(relation_dict)

In [17]:
product_related

,0912696591,B00000JGRP,B00000JGRQ,B00000JGRT,B00002N602,B00004OCIP,B00004OCIQ,B00004OCIU,B00004OCIX,B00004OCIZ,B00004OCJ6,B00004OCJ9,B00004OCJJ,B00004OCJK,B00004OCJN,B00004OCJO,B00004OCJQ,B00004OCJW,B00004OCK0,B00004OCK3,B00004OCKO,B00004OCKR,B00004OCKT,B00004OCL2,B00004OCL8,B00004OCL9,B00004OCLA,B00004OCLK,B00004OCLW,B00004OCM4,B00004OCMB,B00004OCMM,B00004OCMS,B00004OCMX,B00004OCMY,B00004OCNE,B00004OCNJ,B00004OCNK,B00004OCNL,B00004OCNQ,...,B00JW02L9W,B00JWXSPAS,B00JWY1PPE,B00JWYX7YQ,B00JWZCE08,B00JZKZPMO,B00JZL11V2,B00JZXYG9O,B00K11U0Y4,B00K4S1L1K,B00K4V4EWA,B00K6BCV7S,B00K7ILPYU,B00K7P0D9G,B00K8LHA3G,B00KAF5DLQ,B00KAJ3OY0,B00KAK443Y,B00KB7KNRM,B00KBQ1OHQ,B00KBUGQJS,B00KCFDB4U,B00KD83BMS,B00KDV5RZY,B00KFS40N0,B00KI2RQHU,B00KIVD1PC,B00KKYSBUW,B00KM9W2PU,B00KNPVL8W,B00KQCJ0CG,B00KQXDIG4,B00KSRQN0Q,B00KSRSZC0,B00KSRV3UQ,B00KSRXUC0,B00KW1ZDCM,B00KWKQ2MS,B00L8FQZ3M,B00LB18EKK
also_bought,"[B003MC22TU, B003MC22PY, B006UKNSVC, B003MC22Z...",NaN,"[B00004S9CJ, B001D9B2R2, B000TQEDDK, B00005QFK...","[B00004S9D3, 0894803123, 149484205X, 158008219...","[B000ORD8GM, 0965410803, 0764597264, B0000Z6JI...","[B000SSTV8Q, B0071OUJDQ, B00004OCO8, B0001YH1A...","[B00004OCM4, B007WTGR90, B000SOAT0Y, B002XOHZW...","[B00004OCIP, B000Q9CIYG, B0000DAQ93, B000VQOP5...",NaN,"[B00004OCLJ, B00004OCL2, B003M8GMS6, B00004OCL...","[B000A13OEI, B001BB2LMM, B0011EZ7AC, B000A13OE...","[B00421ATIQ, B000FUO3QM, B002CZOSL8, B0037XIZD...","[B00004OCJK, B00004OCIP, B004VLYQGS, B000YDO2M...","[B00004OCIP, B00004OCNS, B00004OCJJ, B0000CDVD...","[B003BEDQL2, B0000E1FDA, B0000D8CAO, B00004OCI...","[B0000DAQ93, B0036X4YOG, B00B6LUAPW, B003AIKP0...","[B0019R7SPS, B00004OCNJ, B007L5ROZQ, B000I0MGK...","[B00004OCIP, B00006IFN8, B00004OCJG, B0001YH1A...","[B0000CDVD8, B0001BMXIU, B00004OCMX, B003L0OYJ...","[B0002YTG5Y, B0002YTG4K, B0002YTG3G, B0007P4J3...","[B0007VO0EE, B0001YH1A2, B001713L5C, B00004OCI...","[B002MQ3SOA, B002DWA6K2, B000OUY2QO, B003L0OOR...","[B000AREB5S, B0088LR592, B002XOHZWC, B007WTGR9...","[B00004OCIZ, B00004OCLJ, B001CRC54O, B001OHV29...","[B00004OCOP, B000KEYD9K, B00004OCOQ, B00004OCO...","[B00004OCNS, B00004OCJK, B0045EPNBI, B00004OCI...","[B007U256D2, B0042FN98Y, B0010L1C28, B0038JE5Y...","[B002HMBVKW, B00004OCIZ, B00004OCLJ, B000FEDCN...","[B001713L84, B00KY4Z1I8, B001CDLKHQ, B00004OCN...","[B00CPNUY7Y, B002XOHZWC, B00004OCIQ, B00095VK7...","[B00004OCMX, B000EURKG8, B0069KDUXO, B00004OCM...","[B0001YH1A2, B00004OCIP, B0000DAQ46, B0007VO0C...","[B00004OCMT, B00004OCME, B00004OCMW, B00004OCM...","[B00004OCMB, B00004OCMP, B0001BMXIU, B00004OCM...","[B001CDP5VS, B000M8YMEU, B00004OCO6, B00004OCO...","[B000QJGRJ8, B007638GOQ, B00A2KDAJQ, B0006Z16R...","[B000QJE48O, B000IYYFIQ, B000KEUKO2, B000JPSI8...","[B000VWECPQ, B000MMRNPG, B00004OCNJ, B000QJE48...","[B000QJE48O, B00004OCNJ, B00005OAXZ, B00004OCN...","[B00004OCNS, B00004OCNT, B00004OCO8, B000JPSI8...",...,"[B00JJOEV9Y, B00M04HZIO, B00L8FDIZU, B00L8FQZ3...","[B00JWYX7YQ, B00JJOEV9Y, B00LGVUUA2, B00L8HA5L...","[B00JJOEV9Y, B00LGVUUA2, B00M04HZIO, B00L8HA5L...","[B00JJOEV9Y, B00JWXSPAS, B00LGVUUA2, B00M04HZI...","[B00JJOEV9Y, B00L8FDIZU, B00L8FQZ3M, B00LGVUUA...","[B00J4FUCAA, B00K8LHA3G, B00HDABAQS, B0000CFOI...","[B00JVOIW3S, B00J93MQ6U, B00DDMILGM, B001OAKME...","[B00JJOEV9Y, B00M04HZIO, B00LGVUUA2, B00L8FDIZ...","[B00D19SSE2, B005TA1IUW, B0041ENPEO, B00D1AXYA...","[B00JJOEV9Y, B00L9KOZBK, B00M04HZIO, B00LSXYC6...","[B00JS0TIVK, B00I3LDJJW, B005OLJEZC, B000QIU7X...","[B00K6BCW5O, B00KNPVL8W, B004ZEMSCK, B00DOL3LW...","[B00KZNOHTW, B00KQBR9FM, B00KQVTX06, B00KHGIK5...","[B00KCFDB4U, B00K8LHA3G, B00JVOIW3S, B00KWXBZP...","[B00K7P0D9G, B00KCFDB4U, B00JZKZPMO, B00JVOIW3...","[B00006IV0U, B003GSKQAW, B006OK7RRO, B00GOPICN...",NaN,NaN,NaN,"[B00K7P0D9G, B00KCFDB4U, B001T4URXG, B0052456U...",NaN,"[B00K7P0D9G, B00EZQ8OXU, B00KL1JZEU, B00KTV6DY...","[B00J4FUCAA, B00J3M78GA, B00KDV5RZY, B00HDABAQ...","[B00J4FUCAA, B00J3M78GA, B00KDUYU30, B00HDABAQ...","[B00JJOEV9Y, B00M04HZIO, B00KWP8ZIM, B00KVE7R0...","[B00JJ

In [64]:
def get_mean_rating(items):
  if train['asin'].isin(items).sum() != 0:
    return train[train['asin'].isin(items)].overall.mean()
  else:
    return 0

In [67]:
def collaborative_related(user_id, item_id):
  """ collaborative-filtering based on mean ratings of related products. """
  if item_id in train.asin.to_list():
    item_condition = train.asin == item_id
    if item_id in product_related.columns:
      related_items_bought = product_related.loc['also_bought', item_id]
      if type(related_items_bought) is list:
        mean_rate = get_mean_rating(related_items_bought)
        if mean_rate == 0:
          return train.loc[item_condition, 'overall'].mean()
        else:
          return mean_rate*0.5+train.loc[item_condition, 'overall'].mean()*0.5
      else:
        return train.loc[item_condition, 'overall'].mean()
    else:
      return train.loc[item_condition, 'overall'].mean()
  else:
    return 5
print('RMSE for collaborative related mean estimation: %s' % evaluate(collaborative_related))

RMSE for collaborative related mean estimation: 0.9833940525288604
